In [2]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/youtoxic_english_1000.csv")

print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '../data/youtoxic_english_1000.csv'

In [ ]:
# 1. Revisar columnas con solo ceros o sin representación significativa
etiquetas = [
    "IsToxic", "IsAbusive", "IsThreat", "IsProvocative", "IsObscene",
    "IsHatespeech", "IsRacist", "IsNationalist", "IsSexist", "IsHomophobic",
    "IsReligiousHate", "IsRadicalism"
]

# Mostrar cuántas veces aparece "True" en cada etiqueta
print("Etiquetas activas:")
print(df[etiquetas].sum().sort_values())

# 2. Eliminar columnas sin representación (sum == 0 o sum == 1)
etiquetas_utiles = [col for col in etiquetas if df[col].sum() > 1]
print("\nEtiquetas que conservaremos:", etiquetas_utiles)

# 3. Eliminar columnas irrelevantes para el modelado
columnas_irrelevantes = ["CommentId", "VideoId"]
df.drop(columns=columnas_irrelevantes, inplace=True)

# 4. (Opcional) Eliminar duplicados por texto
df.drop_duplicates(subset="Text", inplace=True)

# 5. Dejar el DataFrame con solo las columnas útiles
columnas_utiles = ["Text"] + etiquetas_utiles
df = df[columnas_utiles]

# 6. Confirmar
print(f"\nDataset limpio → Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
df.head()


### Limpieza previa del dataset

Antes de aplicar el preprocesamiento textual, revisamos las columnas del dataset:

- Eliminamos etiquetas que **no tienen representación suficiente** (por ejemplo, `IsHomophobic`, `IsRadicalism`) ya que no aportarían valor al modelo.
- Quitamos columnas irrelevantes como `CommentId` y `VideoId`.
- Eliminamos posibles duplicados exactos en los comentarios (`Text`).

Esto nos permite trabajar sobre un dataset más limpio, centrado en los comentarios y en las etiquetas que tienen información útil. De este modo, evitamos introducir ruido o clases vacías en el modelo.


In [ ]:
import spacy
import re

# Cargar modelo de inglés
nlp = spacy.load("en_core_web_sm")

# Función para limpiar y lematizar texto
def preprocesar_texto(texto):
    # 1. Eliminar URLs, símbolos especiales y números
    texto = re.sub(r"http\S+|www\S+|[^a-zA-Z\s]", "", texto.lower())
    
    # 2. Procesar con SpaCy
    doc = nlp(texto)

    # 3. Eliminar stopwords y obtener lemas
    tokens_limpios = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    return " ".join(tokens_limpios)


In [ ]:
# Aplicar limpieza sobre una copia del texto
df["CleanText"] = df["Text"].apply(preprocesar_texto)

# Ver resultado en 3 ejemplos
df[["Text", "CleanText"]].sample(3)


### Preprocesamiento del texto con SpaCy

En este paso limpiamos y normalizamos los comentarios para que puedan ser procesados por modelos de machine learning. Para ello utilizamos **SpaCy**, una librería especializada en procesamiento de lenguaje natural.

Nuestra función `preprocesar_texto()` realiza las siguientes tareas:

1. **Elimina elementos innecesarios** del texto original:
   - URLs, símbolos, números y puntuación.
   - Convierte todo a minúsculas.

2. **Tokeniza** (divide en palabras) y procesa cada palabra con SpaCy:
   - Elimina las **palabras vacías** (*stopwords*) como "the", "you", "and".
   - Obtiene la **forma base** (*lema*) de cada palabra, por ejemplo:
     - "running" → "run"
     - "was" → "be"
     - "better" → "good"

3. **Reconstruye el texto limpio**, que usaremos como entrada para vectorización y modelado.

Este preprocesamiento mejora significativamente la calidad de las características que extraeremos del texto, ya que reduce el ruido y normaliza el lenguaje.


In [ ]:
# Esto puede tardar unos segundos la primera vez (1.000 textos)
df["CleanText"] = df["Text"].apply(preprocesar_texto)

# Comprobar algunos ejemplos de texto original vs limpio
df[["Text", "CleanText"]].sample(5, random_state=42)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Crear los vectorizadores
vectorizer_bow = CountVectorizer(max_features=20)
vectorizer_tfidf = TfidfVectorizer(max_features=20)

# Aplicar sobre el texto limpio
X_bow = vectorizer_bow.fit_transform(df["CleanText"])
X_tfidf = vectorizer_tfidf.fit_transform(df["CleanText"])

# Convertir a DataFrames para verlos
df_bow = pd.DataFrame(X_bow.toarray(), columns=vectorizer_bow.get_feature_names_out())
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

# Mostrar las primeras filas de cada uno
print("Bag of Words:")
display(df_bow.head())

print("TF-IDF:")
display(df_tfidf.head())


### Comparación entre Bag of Words y TF-IDF

Ambas técnicas convierten texto en vectores numéricos, pero lo hacen de forma distinta:

- **Bag of Words** simplemente cuenta cuántas veces aparece cada palabra.
- **TF-IDF** ajusta esos conteos según la rareza de cada palabra en el conjunto total.

#### ¿Qué vemos?
- En BoW, los valores son enteros (frecuencias puras).
- En TF-IDF, los valores son decimales → ajustados por importancia.
- Palabras comunes (como "people") tendrán menos peso en TF-IDF si aparecen en casi todos los comentarios.

Ambas representaciones son útiles, pero **TF-IDF suele funcionar mejor en clasificación** al eliminar ruido estadístico y destacar las palabras que verdaderamente diferencian un texto de otro.


In [ ]:
print("🔤 Vocabulario BoW:")
print(vectorizer_bow.get_feature_names_out())

print("\n🔤 Vocabulario TF-IDF:")
print(vectorizer_tfidf.get_feature_names_out())


In [ ]:
# Sparsity de BoW
sparsity_bow = 1.0 - (X_bow.count_nonzero() / float(X_bow.shape[0] * X_bow.shape[1]))
print(f"Sparsity BoW: {sparsity_bow:.2%}")

# Sparsity de TF-IDF
sparsity_tfidf = 1.0 - (X_tfidf.count_nonzero() / float(X_tfidf.shape[0] * X_tfidf.shape[1]))
print(f"Sparsity TF-IDF: {sparsity_tfidf:.2%}")


### Comparación de vocabulario y sparsity entre BoW y TF-IDF

Después de aplicar las técnicas de vectorización, hemos comparado:

#### El vocabulario generado
Ambos métodos (`CountVectorizer` y `TfidfVectorizer`) seleccionaron las 20 palabras más frecuentes en el dataset. Estas palabras forman el **vocabulario base** sobre el cual se construyen los vectores numéricos de cada comentario.

Aunque el vocabulario es el mismo en este caso (por usar `max_features=20`), las **frecuencias que asignan a cada palabra son diferentes**:

#### Sparsity (dispersión de la matriz)
Calculamos la **sparsity** de ambas matrices, es decir, el porcentaje de celdas con valor cero. El resultado fue:

- Sparsity BoW: **90.52%**
- Sparsity TF-IDF: **90.52%**

Esto significa que más del 90% de los valores en ambas matrices son ceros, lo cual es normal en NLP cuando la mayoría de las palabras **no aparecen en la mayoría de los textos**. Aun así, este tipo de representación dispersa funciona bien en modelos de clasificación textual, especialmente con TF-IDF que reduce el ruido de palabras demasiado comunes.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Dividir datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, df["IsToxic"], test_size=0.2, random_state=42
)

# 2. Entrenar modelo base
modelo_lr = LogisticRegression(max_iter=1000)
modelo_lr.fit(X_train, y_train)

# 3. Predecir sobre el conjunto de test
y_pred = modelo_lr.predict(X_test)

# 4. Métricas
print("📊 Reporte de clasificación:\n")
print(classification_report(y_test, y_pred, digits=3))

# 5. Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Tóxico", "Tóxico"], yticklabels=["No Tóxico", "Tóxico"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("🔍 Matriz de Confusión - Logistic Regression")
plt.show()

# 6. Accuracy global
print(f"✔️ Accuracy del modelo: {accuracy_score(y_test, y_pred):.2%}")


NameError: name 'X_tfidf' is not defined

### Primer modelo de clasificación con TF-IDF + Logistic Regression

En esta celda hemos entrenado nuestro primer modelo de machine learning para detectar comentarios tóxicos utilizando el texto preprocesado y vectorizado con **TF-IDF**.

#### ¿Qué hicimos?
1. **División de datos**
   - Separamos el dataset en un 80% para entrenamiento y 20% para prueba.
   - Esto nos permite evaluar el modelo con ejemplos que nunca ha visto.

2. **Entrenamiento**
   - Usamos un modelo de regresión logística (`LogisticRegression`), ideal como punto de partida en clasificación binaria.

3. **Evaluación**
   - Calculamos métricas como **precisión, recall y F1-score** para ambas clases (`tóxico` y `no tóxico`).
   - Generamos una **matriz de confusión** para visualizar los aciertos y errores del modelo.
   - Mostramos el **accuracy general** del modelo.

#### ¿Qué esperamos aquí?
Este modelo es nuestra línea base (*baseline*). No se ha optimizado aún, pero ya nos permite:
- Ver si el texto limpio y vectorizado con TF-IDF ofrece buena señal.
- Identificar si el modelo tiene sesgos (por ejemplo, si falla mucho en detectar comentarios tóxicos).
- Comparar en el futuro con modelos más complejos.

Este paso marca el comienzo de la fase de modelado y nos da una primera medida de rendimiento que intentaremos superar con técnicas más avanzadas.
